<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/master/MF_model_song_and_tag_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
pip install implicit

     |████████████████████████████████| 1.1MB 6.6MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420097 sha256=950c966c13c7f0a0f71d6a95f1a905c3117c37d0d95923e0a4ad06696c9654fe
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
import scipy
warnings.filterwarnings('ignore')
import implicit

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
folder = '/content/gdrive/My Drive/melon kakao'
filelist = os.chdir(folder)

In [5]:
genre_gn_all=pd.read_json('genre_gn_all.json', typ='series')
train=pd.read_json('train.json')
song_meta=pd.read_json('song_meta.json')
test=pd.read_json('test.json')

플레이리스트 생성 시간보다 곡 발매일이 나중에 나온경우 삭제하기


In [6]:
#starnge song 생성

plylst_date = train[['updt_date', 'songs']]
plylst_date_unnest = np.dstack(
    (
        np.repeat(plylst_date.updt_date.values, list(map(len, plylst_date.songs))), 
        np.concatenate(plylst_date.songs.values)
    )
)
plylst_date = pd.DataFrame(data=plylst_date_unnest[0], columns = plylst_date.columns)

del plylst_date_unnest


plylst_date["updt_date"] = plylst_date["updt_date"].map(lambda x: x[:4] + x[5:7] + x[8:10])
song_date = song_meta[["id","issue_date"]]
plylst_song_date = pd.merge(plylst_date, song_date, left_on="songs", right_on="id", how='left')
plylst_song_date["issue_date"] = plylst_song_date["issue_date"].astype(str)

plylst_song_date["strange"] = plylst_song_date["updt_date"] < plylst_song_date["issue_date"]
strange_songs = plylst_song_date[plylst_song_date["strange"] == True].drop('songs', axis=1)
strange_songs=strange_songs.drop_duplicates(['id'])

strange_songs_list=strange_songs['id'].values.astype(str)


#train dataframe 전처리

#플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map=train[['id', 'songs']]

#unnest songs

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

#unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype(str)
plylst_song_map['songs']=plylst_song_map['songs'].astype(str)

del plylst_song_map_unnest

plylst_song_map_remove=plylst_song_map[np.logical_not(plylst_song_map.songs.isin(strange_songs_list))]


plylst_song_group=plylst_song_map_remove.groupby('id')['songs'].apply(list).reset_index(name='remove_songs')
train['id']=train['id'].astype(str)
train_remove = pd.merge(train, plylst_song_group)
train_remove['songs']=train_remove['remove_songs']
train_remove=train_remove.drop('remove_songs', axis=1)

matrix factorization model 세우기 - song prediction

In [7]:
#songs 존재하는 test 데이터 남기기

train_songs=train[['id', 'songs']]
test_songs=test[['id', 'songs']]


filter=test_songs['songs'].astype('str')=='[]'
test_songs=test_songs[~filter]

plylst_song_map=pd.concat([train_songs, test_songs])


#plylst_song_map unnest

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype('category')
plylst_song_map['songs']=plylst_song_map['songs'].astype('category')

In [8]:
#sparse matrix 만들기

sparse_song=scipy.sparse.coo_matrix((np.ones(plylst_song_map.shape[0]),
                                (plylst_song_map['songs'].cat.codes.copy(),
                                 plylst_song_map['id'].cat.codes.copy())))

In [9]:
#ALS model 세우기

model_song=implicit.als.AlternatingLeastSquares(factors=10,  regularization=0.01, dtype=np.float64, iterations=20, use_gpu=False)
model_song.fit(sparse_song)

In [10]:
#recommendation

repos_song=dict(enumerate(plylst_song_map['songs'].cat.categories))

plylst_song_map_test=test_songs

#test set unnest

plylst_song_map_test_unnest=np.dstack(
    (
        np.repeat(plylst_song_map_test.id.values, list(map(len, plylst_song_map_test.songs))),
        np.concatenate(plylst_song_map_test.songs.values)
    )
)

plylst_song_map_test=pd.DataFrame(data=plylst_song_map_test_unnest[0], columns=plylst_song_map_test.columns)
plylst_song_map_test['id']=plylst_song_map_test['id'].astype('category')
plylst_song_map_test['songs']=plylst_song_map_test['songs'].astype('category')

del plylst_song_map_test_unnest

#test_song data에 대한 sparse matrix 만들기

test_song_sparse=scipy.sparse.coo_matrix((np.ones(plylst_song_map_test.shape[0]),
                                          (plylst_song_map_test['id'].cat.codes.copy(),
                                           plylst_song_map_test['songs'].cat.codes.copy()))).tocsr()

In [11]:
result_songs = []

for i in range(test_songs.shape[0]):
  rec=model_song.recommend(userid=i, user_items=test_song_sparse, N=100)
  result_rec=[]
  for j in range(len(rec)):
    rec[j]=repos_song[rec[j][0]]
    result_rec.insert(j, rec[j])
  result_songs.insert(i, result_rec)


In [12]:
test_songs['predicted_songs']=result_songs

song data를 이용하여 song 예측하기

In [13]:
#곡 데이터를 이용하여 곡 prediction 결과

test_result=test_songs.drop('songs', axis=1)
print(test_result)

test_result.to_json('J_result_1.json')

           id                                    predicted_songs
0       70107  [581799, 493762, 15124, 195174, 191915, 146989...
1        7461  [581799, 493762, 15124, 195174, 146989, 191915...
2       90348  [581799, 493762, 15124, 146989, 625875, 195174...
3       58617  [133143, 650494, 701557, 610933, 132994, 34820...
4      102395  [116573, 675115, 654757, 37748, 88503, 224921,...
...       ...                                                ...
10735  137930  [133143, 650494, 701557, 610933, 132994, 34820...
10736     936  [144663, 116573, 493762, 357367, 37748, 581799...
10737  110589  [654757, 117595, 88503, 1236, 113958, 173943, ...
10738    2605  [581799, 493762, 237407, 15124, 505036, 195174...
10739  126651  [519391, 640657, 133484, 698536, 232606, 11228...

[8697 rows x 2 columns]


태그 데이터 이용하여 태그 예측하기

In [14]:
train_tag=train[['id', 'tags']]
test_tag=test[['id', 'tags']]
filter=test_tag['tags'].astype('str')=='[]'
test_tag=test_tag[~filter]


plylst_tag_map=pd.concat([train_tag, test_tag])

#unnesting

plylst_tag_map_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))),
        np.concatenate(plylst_tag_map.tags.values)
    )
)

plylst_tag_map=pd.DataFrame(data=plylst_tag_map_unnest[0], columns=plylst_tag_map.columns)
plylst_tag_map['id']=plylst_tag_map['id'].astype('category')
plylst_tag_map['tags']=plylst_tag_map['tags'].astype('category')

del plylst_tag_map_unnest

In [15]:
#sparse matrix 만들기

sparse_tag=scipy.sparse.coo_matrix((np.ones(plylst_tag_map.shape[0]),
                                    (plylst_tag_map['tags'].cat.codes.copy(),
                                     plylst_tag_map['id'].cat.codes.copy())))

In [16]:
#모델 세우기

model_tag=implicit.als.AlternatingLeastSquares(factors=20,  regularization=0.01, dtype=np.float64, iterations=20, use_gpu=False)
model_tag.fit(sparse_tag)

In [17]:
#recommendation

repos_tags=dict(enumerate(plylst_tag_map['tags'].cat.categories))


plylst_tag_map_test=test_tag

#unnesting

plylst_tag_map_test_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map_test.id.values, list(map(len, plylst_tag_map_test.tags))),
        np.concatenate(plylst_tag_map_test.tags.values)
    )
)


plylst_tag_map_test=pd.DataFrame(data=plylst_tag_map_test_unnest[0], columns=plylst_tag_map_test.columns)
plylst_tag_map_test['id']=plylst_tag_map_test['id'].astype('category')
plylst_tag_map_test['tags']=plylst_tag_map_test['tags'].astype('category')


In [18]:

#test sparse matrix

test_tag_sparse=scipy.sparse.coo_matrix((np.ones(plylst_tag_map_test.shape[0]),
                                          (plylst_tag_map_test['id'].cat.codes.copy(),
                                           plylst_tag_map_test['tags'].cat.codes.copy()))).tocsr()

In [19]:
result_tag=[]

for i in range(test_tag.shape[0]):
  rec=model_tag.recommend(i, test_tag_sparse, N=10)
  result_rec=[]
  for j in range(len(rec)):
    rec[j]=repos_tags[rec[j][0]]
    result_rec.insert(j, rec[j])
  result_tag.insert(i, result_rec)


In [20]:
test_result_tag=test_tag
test_result_tag['predicted_tags']=result_tag
test_result_tag=test_result_tag.drop('tags', axis=1).reset_index()

In [22]:
test_result_tag.to_json('mf_tag.json')